In [12]:
import io
from skbio import DNA, RNA, Sequence
import pandas as pd
from Bio import Entrez

## ENTER EMAIL ADDRESS
Entrez.email = ''

In [13]:
## UPDATE PATHS
gbff_dir = ""
output_file = ""

In [14]:
files = !find "$gbff_dir" -name "*.gbff"

master_dict = {}
for item in files:
    name = item.split('/')[-1][:-5]
    with open(item, 'r') as myfile:
        gb_str = myfile.read()
        gb = io.StringIO(gb_str)
        dna_seq = DNA.read(gb)
        metadata_dict = dna_seq.metadata
        count = 0
        for item in dna_seq.interval_metadata.query(bounds=None, metadata={'type': 'source'}): 
            item_metadata = item.metadata
    metadata_dict.update(item_metadata)
    master_dict[name] = metadata_dict
    

In [15]:
db_dict = {}

strain_list = []
data = []
cols = []
attr = ['strain', 'serotype', 'serovar', 'isolation_source', 'country', 'state', 'collection_date', 'type_material', 'culture_collection', 'note', 'concat_pub']

for key, value in master_dict.items(): 
    master_list = []
    concat_pub = " "
    count = 0 
    for item in value['REFERENCE']: 
        count += 1
        author = item.get('AUTHORS', "")
        consrtm = item.get('CONSRTM', "")
        title = item.get('TITLE', "")
        journal = item.get("JOURNAL", "")
        pub_str = str(count) + '. ' + author + " " + consrtm + " " + title + ". " + journal + ". "
        concat_pub = (concat_pub + pub_str).replace('"', '')
    serotype = str(value.get('serotype', 'NA')).replace('"', '')
    isolation_source = str(value.get('isolation_source', 'NA').replace('"', ''))
    country = str(value.get('country', 'NA').replace('"', '')).split(':')[0]
    collection_date = str(value.get('collection_date', 'NA').replace('"', ''))
    note = str(value.get('note', 'NA').replace('"', ''))
    type_material = str(value.get('type_material', 'NA').replace('"', ''))
    strain = str(value.get('strain', 'NA').replace('"', ''))
    serovar = str(value.get('serovar', 'NA').replace('"',''))
    serotype = str(value.get('serotype', 'NA').replace('"',''))
    culture_collection = str(value.get('culture_collection', 'NA').replace('"',''))
    if len(value.get('country', 'NA').replace('"', '').split(':'))> 1: 
        state = str(value.get('country', 'NA').replace('"', '')).replace(" ", '').split(':')[1]
    else: 
        state = "NA"
    strain_list.append(key)
    master_list.append(strain)
    master_list.append(serotype)
    master_list.append(serovar)
    master_list.append(isolation_source)
    master_list.append(country)
    master_list.append(state)
    master_list.append(collection_date)
    master_list.append(type_material)
    master_list.append(culture_collection)
    master_list.append(note)
    master_list.append(concat_pub.replace('"', ""))
    data.append(master_list) 
    

In [16]:
df = pd.DataFrame(data, index= strain_list, columns= attr)
pd.DataFrame.to_csv(df, output_file)